In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 8
fig_height = 4
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/diego/local_repos/AGEC652_2024/slides/lecture_7_1"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| include: false
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# Pkg.add("JuMP")
# Pkg.add("Ipopt")
# Pkg.add("Statistics")
# Pkg.add("Plots")
# Pkg.add("ForwardDiff")
# Pkg.add("LinearAlgebra")

  Activating project at `~/local_repos/AGEC652_2024/slides/lecture_7_1`


In [3]:
using Statistics, Random
# Generate some synthetic data for illustration
Random.seed!(652) # Set random seed for reproducibility
N = 1000  # Number of observations
X = rand(N) * 10  # Independent variable: random uniform [0,10]
β0_true = 2.0  # True intercept
β1_true = 3.0  # True slope
σ_true = 1.5  # True standard deviation of the errors
ε = randn(N) * σ_true  # Normally distributed errors
Y = β0_true .+ β1_true .* X .+ ε  # Dependent variable

1000-element Vector{Float64}:
 21.67019447290482
 28.90711926428413
 16.442940381636067
 22.364903307770113
 16.223008723288245
  9.150914155735176
  6.494370868789826
 29.190769080962113
  8.845913560652406
 19.768418811032205
  8.768806341579456
 28.905029646813723
 12.30335776513745
  ⋮
 11.360611536318569
 19.1496946598927
 25.457203383492395
  1.4494239993093165
  9.673970881293892
 20.211214574506702
  9.796687093954356
 24.24916530921735
 17.096168925423544
  5.155534028613376
 17.977179360988384
 16.455682148662746

In [4]:
function g_i(θ; Y = Y, X = X)
  β_0, β_1 = θ # Unpack parameters
  ϵ = Y .- β_0 .- β_1 .* X
  return hcat(ϵ, X.*ϵ) # Concatenate horizontally
end;

In [5]:
G = g_i([0.0, 0.0], Y=Y, X=X);
G[1:6,:] # See the first few lines

6×2 Matrix{Float64}:
 21.6702   133.22
 28.9071   276.875
 16.4429    78.751
 22.3649   169.115
 16.223     71.2158
  9.15091   18.6789

In [6]:
function g_N(θ; Y = Y, X = X)
  N = length(Y) # Number of obs
  # Get moment vectors
  g_Ns = g_i(θ; Y = Y, X = X)
  # Take means of each column and return
  return [sum(g_Ns[:, k]) for k in 1:2] ./ N
end;

In [7]:
g_N([0.0, 0.0], Y=Y, X=X)

2-element Vector{Float64}:
  16.91885348772398
 109.2561898066229

In [8]:
using LinearAlgebra
function Q_N(θ; W = I(2), Y = Y, X = X)
    return g_N(θ; Y=Y, X=X)' * W * g_N(θ; Y=Y, X=X)
end;

In [9]:
θ_0 = [0.0, 0.0] # Initial guess
W_0 = I(2) # 2x2 identity matrix
Q_N([0.0, 0.0], W=W_0, Y=Y, X=X)

12223.16261439988

In [10]:
using Optim
res_step_1 = Optim.optimize(θ -> Q_N(θ; W = W_0), θ_0, Newton())
θ_1 = res_step_1.minimizer # Step 1 estimate

2-element Vector{Float64}:
 2.115784665111538
 2.971465985897643

In [11]:
W_hat = inv(g_i(θ_1)'  * g_i(θ_1) ./N)

2×2 Matrix{Float64}:
  1.78631  -0.2701
 -0.2701    0.054656

In [12]:
res_step_2 = Optim.optimize(θ -> Q_N(θ; W = W_hat), θ_0, Newton())
θ_2 = res_step_2.minimizer # Step 2 estimate

2-element Vector{Float64}:
 2.115784665111508
 2.9714659858976464

In [13]:
using ForwardDiff
D_hat = ForwardDiff.jacobian(g_N, θ_2)

2×2 Matrix{Float64}:
 -1.0       -4.98174
 -4.98174  -33.2213

In [14]:
S_hat = g_i(θ_2)'  * g_i(θ_2) ./N

2×2 Matrix{Float64}:
  2.21471  10.9447
 10.9447   72.3829

In [15]:
V_hat = inv(D_hat' * inv(S_hat) * D_hat) ./N

2×2 Matrix{Float64}:
  0.00875061  -0.00130145
 -0.00130145   0.000259132

In [16]:
SEs = sqrt.(diag(V_hat))

2-element Vector{Float64}:
 0.09354468036028733
 0.016097563221981737

In [17]:
using DataFrames, Distributions
df = DataFrame(
  Coefficient = ["beta_0", "beta_1"],
  Estimate = θ_2,
  StdError = SEs,
  CI_lower = θ_2 .+ quantile(Normal(), 0.025) .* SEs,
  CI_upper = θ_2 .+ quantile(Normal(), 0.975) .* SEs
)

println(df)

2×5 DataFrame
 Row │ Coefficient  Estimate  StdError   CI_lower  CI_upper 
     │ String       Float64   Float64    Float64   Float64  
─────┼──────────────────────────────────────────────────────
   1 │ beta_0        2.11578  0.0935447   1.93244   2.29913
   2 │ beta_1        2.97147  0.0160976   2.93992   3.00302
